In [1]:
val sparksession=new SparkSession.Builder().appName("ratingsession").getOrCreate()

sparksession = org.apache.spark.sql.SparkSession@40a61e51


org.apache.spark.sql.SparkSession@40a61e51

In [3]:
val ratings_csv_path="gs://movielens_dataset_rd/ratings.csv"
val ratingsDfFull=sparksession.read.option("header", "true").option("inferSchema", "true").csv(ratings_csv_path)


ratings_csv_path = gs://movielens_dataset_rd/ratings.csv
ratingsDfFull = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [4]:
val ratingsDf=ratingsDfFull.limit(10000)

ratingsDf = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [6]:
import org.apache.spark.sql.functions._

In [7]:
val ratingsDfWithYear = ratingsDf
  .withColumn("year", year(from_unixtime(col("timestamp"))))
  

ratingsDfWithYear = [userId: int, movieId: int ... 3 more fields]


[userId: int, movieId: int ... 3 more fields]

In [8]:
val finalRatingsDf=ratingsDfWithYear.select("userId","movieId","rating","year")

finalRatingsDf = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [9]:
finalRatingsDf.limit(5).show()

+------+-------+------+----+
|userId|movieId|rating|year|
+------+-------+------+----+
|     1|     17|   4.0|1999|
|     1|     25|   1.0|1999|
|     1|     29|   2.0|1999|
|     1|     30|   5.0|1999|
|     1|     32|   5.0|1999|
+------+-------+------+----+



In [10]:
val ratingsRdd=finalRatingsDf.rdd

MapPartitionsRDD[25] at rdd at <console>:27

ratingsRdd = MapPartitionsRDD[25] at rdd at <console>:27


In [16]:
val grouppedRdd=ratingsRdd.groupBy(rating=>rating.getAs[Integer]("year"))

grouppedRdd = ShuffledRDD[29] at groupBy at <console>:27


ShuffledRDD[29] at groupBy at <console>:27

In [14]:
grouppedRdd.take(5).foreach(println)

(2021,CompactBuffer([22,316,3.0,2021], [22,5049,2.5,2021], [22,155509,4.0,2021], [22,182823,3.5,2021], [22,191489,3.5,2021], [22,197889,4.0,2021], [22,210271,2.5,2021], [22,218867,3.0,2021], [22,235509,2.5,2021], [22,247150,3.0,2021], [23,2890,4.0,2021], [23,45672,5.0,2021], [23,110110,5.0,2021], [23,112183,5.0,2021], [23,113348,4.5,2021], [23,138036,5.0,2021], [23,142488,5.0,2021], [23,167544,4.5,2021], [23,210861,5.0,2021], [23,219444,4.5,2021], [23,250010,5.0,2021], [28,1209,4.5,2021], [28,6350,4.0,2021], [28,26776,4.0,2021], [28,31658,4.5,2021], [28,65261,4.5,2021], [28,70533,3.5,2021], [28,84187,4.0,2021], [28,90439,3.0,2021], [28,101962,4.0,2021], [28,103228,3.0,2021], [28,116975,4.0,2021], [28,135143,4.0,2021], [28,163134,5.0,2021], [28,166291,5.0,2021], [28,191459,3.5,2021], [28,195163,3.5,2021], [28,196891,3.5,2021], [28,202429,4.0,2021], [28,202439,5.0,2021], [28,203160,4.5,2021], [28,210569,3.5,2021], [28,210851,4.0,2021], [28,218537,4.0,2021], [28,223876,4.0,2021], [28,2310

In [ ]:
val listRdds=groupedRdd.map()

In [37]:
grouppedRdd.collect.foreach({case(year,ratings)=>{
    val outputDir = s"hdfs:////output/caseStudy5/output/user-data/$year/ratings.parquet"
    
//     val flattenedRatings=ratings.flatMap(rating=>rating)
    
    
    val ratingsRd=sparksession.sparkContext.parallelize(ratings.toSeq)
    
    
    
     
    val tempDf=sparksession.createDataFrame(ratingsRd,finalRatingsDf.schema)
    
    tempDf.write.parquet(outputDir)
    
    
    
    
//     ratings.save(outputDir)
}})

In [23]:
val test=grouppedRdd.take(1)


test = Array((2021,CompactBuffer([22,316,3.0,2021], [22,5049,2.5,2021], [22,155509,4.0,2021], [22,182823,3.5,2021], [22,191489,3.5,2021], [22,197889,4.0,2021], [22,210271,2.5,2021], [22,218867,3.0,2021], [22,235509,2.5,2021], [22,247150,3.0,2021], [23,2890,4.0,2021], [23,45672,5.0,2021], [23,110110,5.0,2021], [23,112183,5.0,2021], [23,113348,4.5,2021], [23,138036,5.0,2021], [23,142488,5.0,2021], [23,167544,4.5,2021], [23,210861,5.0,2021], [23,219444,4.5,2021], [23,250010,5.0,2021], [28,1209,4.5,2021], [28,6350,4.0,2021], [28,26776,4.0,2021], [28,31658,4.5,2021], [28,65261,4.5,2021], [28,70533,3.5,2021], [28,84187,4.0,2021], [28,90439,3.0,2021], [28,101962,4.0,2021], [28,103228,3.0,2021], [28,116975,4.0,2021], [28,135143,4.0,2021], [2...


Array((2021,CompactBuffer([22,316,3.0,2021], [22,5049,2.5,2021], [22,155509,4.0,2021], [22,182823,3.5,2021], [22,191489,3.5,2021], [22,197889,4.0,2021], [22,210271,2.5,2021], [22,218867,3.0,2021], [22,235509,2.5,2021], [22,247150,3.0,2021], [23,2890,4.0,2021], [23,45672,5.0,2021], [23,110110,5.0,2021], [23,112183,5.0,2021], [23,113348,4.5,2021], [23,138036,5.0,2021], [23,142488,5.0,2021], [23,167544,4.5,2021], [23,210861,5.0,2021], [23,219444,4.5,2021], [23,250010,5.0,2021], [28,1209,4.5,2021], [28,6350,4.0,2021], [28,26776,4.0,2021], [28,31658,4.5,2021], [28,65261,4.5,2021], [28,70533,3.5,2021], [28,84187,4.0,2021], [28,90439,3.0,2021], [28,101962,4.0,2021], [28,103228,3.0,2021], [28,116975,4.0,2021], [28,135143,4.0,2021], [2...

In [26]:
test.map({case (year,ratings)=>{
    ratings.map(rating=>println(rating.))
}})

Unknown Error: <console>:29: error: value _1 is not a member of org.apache.spark.sql.Row
           ratings.map(rating=>println(rating._1))
                                              ^
